In [1]:
import ctypes
import numpy as np

In [2]:
a = np.load("triangles_a.npy")
b = np.load("triangles_b.npy")

In [3]:
ifortlib = ctypes.CDLL("./src/fortran/ifort_clipping.dll")
def ifort_area_of_intersection(a, b):
    ntriangles, nvertex, ndim = a.shape
    out = np.empty(ntriangles, dtype=np.float64)
    function = ifortlib.area_of_intersection
    function(
        ctypes.pointer(ctypes.c_longlong(ndim)),
        ctypes.pointer(ctypes.c_longlong(nvertex)),
        ctypes.pointer(ctypes.c_longlong(ntriangles)),
        np.ctypeslib.as_ctypes(a),
        np.ctypeslib.as_ctypes(b),
        np.ctypeslib.as_ctypes(out),
    )
    return out

gfortranlib = ctypes.CDLL("./src/fortran/gfortran_clipping.dll")
def gfortran_area_of_intersection(a, b):
    ntriangles, nvertex, ndim = a.shape
    out = np.empty(ntriangles, dtype=np.float64)
    function = gfortranlib.area_of_intersection
    function(
        ctypes.pointer(ctypes.c_longlong(ndim)),
        ctypes.pointer(ctypes.c_longlong(nvertex)),
        ctypes.pointer(ctypes.c_longlong(ntriangles)),
        np.ctypeslib.as_ctypes(a),
        np.ctypeslib.as_ctypes(b),
        np.ctypeslib.as_ctypes(out),
    )
    return out

In [4]:
%timeit ifort_area_of_intersection(a, b)

115 ms ± 1.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%timeit gfortran_area_of_intersection(a, b)

214 ms ± 3.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
